In [3]:
import MatrixLib
from MatrixLib import FullMatrix, Matrix
from importlib import reload
import numpy as np
# reload(MatrixLib)


In [2]:
m = FullMatrix.zero(3,5,0)
print(m)
print(m.shape)
print(m.dtype)

| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
(3, 5)
int32


In [5]:
m = FullMatrix(np.array([[2,2], [1,2]]))
m

| 2 2 |
| 1 2 |

In [3]:
a, b = m.lu()
a, b

(array([[1. , 0. ],
        [0.5, 1. ]]),
 array([[2., 2.],
        [0., 1.]]))

In [4]:
np.dot(a,b)

array([[2., 2.],
       [1., 2.]])

In [5]:
m.det()

2.0

In [18]:
m1 = FullMatrix(np.array([[2,2], [1,2]]))
m2 = FullMatrix(np.array([[2,3], [3,2]]))
m1*m2

array([[10., 10.],
       [ 8.,  7.]])

In [18]:
from fractions import Fraction
d = FullMatrix.zero(2,2,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)  
        
d

| 0 0 |
| 0 1 |

In [19]:
d.lu()

(0 - 0)/ 0.0 = nan


(array([[ 1.,  0.],
        [nan,  1.]]),
 array([[ 0.,  0.],
        [ 0., nan]]))

In [25]:
from fractions import Fraction
m = FullMatrix.zero(4,4,Fraction(0,1))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)
d = FullMatrix.zero(4,4,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)        
print(m)
print(d)
print("m+m:")
# print(m+d)
print("m*d:")
print(m*d)

| 1 1/2 1/3 1/4 |
| 2 1   2/3 1/2 |
| 3 3/2 1   3/4 |
| 4 2   4/3 1   |
| 0 0 0 0 |
| 0 1 0 0 |
| 0 0 2 0 |
| 0 0 0 3 |
m+m:
m*d:
[[0.         0.5        0.66666667 0.75      ]
 [0.         1.         1.33333333 1.5       ]
 [0.         1.5        2.         2.25      ]
 [0.         2.         2.66666667 3.        ]]


In [15]:
m.lu()

(2 - 0)/ 1.0 = 2.0
(3 - 0)/ 1.0 = 3.0
(3/2 - 1.5)/ 0.0 = nan
(4 - 0)/ 1.0 = 4.0
(2 - 2.0)/ 0.0 = nan
(4/3 - nan)/ nan = nan


(array([[ 1.,  0.,  0.,  0.],
        [ 2.,  1.,  0.,  0.],
        [ 3., nan,  1.,  0.],
        [ 4., nan, nan,  1.]]),
 array([[1.        , 0.5       , 0.33333333, 0.25      ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        ,        nan,        nan],
        [0.        , 0.        , 0.        ,        nan]]))

In [6]:
b = FullMatrix.zero(3,3,m)
print(b)

| | 2 2 | | 2 2 | | 2 2 | |
| | 1 2 | | 1 2 | | 1 2 | |
|                         |
| | 2 2 | | 2 2 | | 2 2 | |
| | 1 2 | | 1 2 | | 1 2 | |
|                         |
| | 2 2 | | 2 2 | | 2 2 | |
| | 1 2 | | 1 2 | | 1 2 | |


In [7]:
b.lu()

TypeError: unsupported operand type(s) for -: 'FullMatrix' and 'NoneType'